In [1]:
import pandas as pd
import numpy as np
import math
import os
from matplotlib import pyplot as plt
from PyRadioLoc.Utils.GeoUtils import GeoUtils
%run UtilsLightGBMFingerPrinting.ipynb

In [2]:
FILES = [
    
    #k = 1 LightGBM
    './FinalResult/Resultados_Metodo_28.csv',  # CDB gerado com LightGBM dados Outdoor
    './FinalResult/Resultados_Metodo_29.csv',  # CDB gerado com LightGBM dados Indoor
    './FinalResult/Resultados_Metodo_30.csv',  # CDB gerado com LightGBM dados Indoor-Outdoor
    
    #k = 2 LightGBM
    './FinalResult/Resultados_Metodo_31.csv',  # CDB gerado com LightGBM dados Outdoor
    './FinalResult/Resultados_Metodo_32.csv',  # CDB gerado com LightGBM dados Indoor
    './FinalResult/Resultados_Metodo_33.csv',  # CDB gerado com LightGBM dados Indoor-Outdoor
    
    #k = 3 LightGBM
    './FinalResult/Resultados_Metodo_34.csv',  # CDB gerado com LightGBM dados Outdoor
    './FinalResult/Resultados_Metodo_35.csv',  # CDB gerado com LightGBM dados Indoor
    './FinalResult/Resultados_Metodo_36.csv',  # CDB gerado com LightGBM dados Indoor-Outdoor
    
    #k = 4 LightGBM
    './FinalResult/Resultados_Metodo_37.csv',  # CDB gerado com LightGBM dados Outdoor
    './FinalResult/Resultados_Metodo_38.csv',  # CDB gerado com LightGBM dados Indoor
    './FinalResult/Resultados_Metodo_39.csv',  # CDB gerado com LightGBM dados Indoor-Outdoor
    
    #k = 5 LightGBM
    './FinalResult/Resultados_Metodo_40.csv',  # CDB gerado com LightGBM dados Outdoor
    './FinalResult/Resultados_Metodo_41.csv',  # CDB gerado com LightGBM dados Indoor
    './FinalResult/Resultados_Metodo_42.csv',  # CDB gerado com LightGBM dados Indoor-Outdoor
    
    
]
METHODS = [x.split("_")[-1].split(".")[0] for x in FILES]     # lista contendo os metodos que serão usados
db = pd.read_csv('databases/meds_pandas.csv')
erbs = pd.read_csv('databases/Bts.csv')
#errorLocation = pd.read_csv('result/erros.csv')
ERROS_CSV_FILE = './result-k-fold/erros-LightGBM-k-fold.csv' 

In [3]:
# Gera o dataframe contendo os erros
errors_df = calculate_theorical_errors()
round(errors_df.describe(), 2)

,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
count,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00
mean,98.68,48.15,73.78,89.86,61.54,80.05,94.08,32.11,64.19,100.54,67.67,81.85,93.58,66.48,81.89
std,53.24,19.33,48.73,57.67,18.66,44.56,55.17,19.21,53.11,56.53,19.81,45.84,55.87,20.67,46.66
min,1.88,10.05,6.51,2.16,27.21,8.21,1.17,12.34,1.21,4.01,24.64,1.47,5.58,39.79,5.79
25%,54.85,33.79,42.14,41.61,49.80,50.37,49.47,20.11,27.43,55.97,51.79,51.79,50.45,50.63,50.63
50%,92.76,52.13,57.76,83.70,59.19,70.25,84.90,27.43,39.25,93.36,72.18,72.18,82.64,61.04,68.14
75%,133.80,56.86,97.82,128.64,70.50,98.17,131.30,36.41,90.59,136.84,83.38,98.01,131.55,80.04,99.32
max,251.37,88.24,216.28,285.67,96.41,260.88,261.19,83.90,236.37,254.24,98.01,232.26,295.84,99.32,229.93


# LightGBM-Outdoor:

In [21]:
test_erro = round(errors_df.describe(), 2)
test_erros = errors_df
mean_outdoor = (test_erros['28'] + test_erros['31'] + test_erros['34'] + test_erros['37'] + test_erros['40'])/5
mean_outdoor.head()

0     76.092
1     73.860
2     95.916
3    109.602
4    101.936
dtype: float64

In [22]:
min_outdoor = min(test_erro['28']['min'], test_erro['31']['min'], test_erro['34']['min'], test_erro['37']['min'], test_erro['40']['min'])
min_outdoor

1.17

In [23]:
max_outdoor = max(test_erro['28']['max'], test_erro['31']['max'], test_erro['34']['max'], test_erro['37']['max'], test_erro['40']['max'])
max_outdoor

295.84

# LightGBM-Indoor:

In [25]:
mean_indoor = (test_erros['29'] + test_erros['32'] + test_erros['35'] + test_erros['38'] + test_erros['41'])/5
mean_indoor.head()

0    65.024
1    55.050
2    65.024
3    71.572
4    42.250
dtype: float64

In [26]:
min_outdoor = min(test_erro['29']['min'], test_erro['32']['min'], test_erro['35']['min'], test_erro['38']['min'], test_erro['41']['min'])
min_outdoor

10.05

In [27]:
max_outdoor = max(test_erro['29']['max'], test_erro['32']['max'], test_erro['35']['max'], test_erro['38']['max'], test_erro['41']['max'])
max_outdoor

99.32

# LightGBM-Indoor-Outdoor:

In [29]:
mean_indoor_outdoor = (test_erros['30'] + test_erros['33'] + test_erros['36'] + test_erros['39'] + test_erros['42'])/5
mean_indoor_outdoor.head()

0    114.144
1    100.446
2    119.458
3     79.618
4    139.062
dtype: float64

In [30]:
min_outdoor = min(test_erro['30']['min'], test_erro['33']['min'], test_erro['36']['min'], test_erro['39']['min'], test_erro['42']['min'])
min_outdoor

1.21

In [31]:
max_outdoor = max(test_erro['30']['max'], test_erro['33']['max'], test_erro['36']['max'], test_erro['39']['max'], test_erro['42']['max'])
max_outdoor

260.88

In [32]:
#errors_df.to_csv(ERROS_CSV_FILE, index=False)

In [33]:
erros_LightGBM = [mean_outdoor,mean_indoor, mean_indoor_outdoor ]
erros_LightGBM = pd.DataFrame(erros_LightGBM)
erros_LightGBM = erros_LightGBM.T
erros_LightGBM = round(erros_LightGBM,2)
erros_LightGBM.to_csv(ERROS_CSV_FILE, index=False)
erros_LightGBM.head()

,0,1,2
0,76.09,65.02,114.14
1,73.86,55.05,100.45
2,95.92,65.02,119.46
3,109.60,71.57,79.62
4,101.94,42.25,139.06


In [36]:
results_LightGBM = round(erros_LightGBM.describe(), 2)
results_LightGBM 
#results_LightGBM.to_csv('./result-k-fold/Result-LightGBM-k-fold.csv', index=False)

,0,1,2
count,400.00,400.00,400.00
mean,95.35,55.19,76.35
std,22.93,12.35,29.45
min,33.35,30.13,30.13
25%,77.81,45.57,56.77
50%,95.85,58.92,67.26
75%,110.67,65.02,94.83
max,163.34,71.57,179.47
